In [1]:
from detectron2.utils.logger import setup_logger
setup_logger()

import os

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer

os.chdir(os.path.dirname(os.path.realpath("__file__")))

In [2]:
from detectron2.data.datasets import register_coco_instances
# coco format dataset을 등록합니다. 
# 입력 파라메터 설명: ("데이터셋 이름", "메타데이터", "json 경로", "이미지들의 경로")
register_coco_instances("Duck-Farm-train", {}, "../datasets/Duck-Farm/annotations/labels_train.json", "../datasets/Duck-Farm/train2017")
register_coco_instances("Duck-Farm-val", {}, "../datasets/Duck-Farm/annotations/labels_val.json", "../datasets/Duck-Farm/val2017")

In [3]:
# config 객체를 선언하고 불러옵니다.
cfg = get_cfg()
# cfg.merge_from_list(['MODEL.DEVICE','cpu'])  # For cpu mode.
# Detectron2에서 사전 정의한 retinanet config 파일을 불러옵니다.
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_1x.yaml"))
# 학습 데이터셋, 테스트 데이터셋
cfg.DATASETS.TRAIN = ("Duck-Farm-train",)
cfg.DATASETS.TEST = ("Duck-Farm-val",)
# 데이터를 불러오는 loader의 수를 정합니다. 일반적으로 'cpu 코어의 수 / 2'만큼 설정합니다. 큰 영향은 끼치지 않는 것 같습니다.
cfg.DATALOADER.NUM_WORKERS =  4
# pretrained checkpoint를 불러옵니다.
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x")
# batch size입니다.
cfg.SOLVER.IMS_PER_BATCH = 4
# 학습 수행 시의 iteration 횟수입니다.
cfg.SOLVER.MAX_ITER = 3000
# checkpoint를 저장하는 iteration 간격을 설정합니다.
cfg.SOLVER.CHECKPOINT_PERIOD = 50
# base learning rate를 설정합니다.
cfg.SOLVER.BASE_LR = 0.001  
# WarmupMultiStepLR의 step을 설정합니다. 적용하지 않기를 원하기 때문에 빈 list를 넣었습니다.
cfg.SOLVER.STEPS = []  # do not decay learning rate
# retinanet이 추론할 class의 수를 설정합니다.
cfg.MODEL.RETINANET.NUM_CLASSES = 3

Loading config /home/koowater/.local/lib/python3.8/site-packages/detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [4]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# config 파일을 토대로 trainer를 불러옵니다.
trainer = DefaultTrainer(cfg)
# 가장 최근의 checkpoint를 불러오거나(resume=True인 경우) config 파일의 weights 링크로부터 weights를 불러옵니다.
trainer.resume_or_load(cfg)
# 학습을 진행합니다.
trainer.train()

[04/05 21:59:33 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

Skip loading parameter 'head.cls_score.weight' to the model due to incompatible shapes: (720, 256, 3, 3) in the checkpoint but (27, 256, 3, 3) in the model! You might want to double check if this is expected.
Skip loading parameter 'head.cls_score.bias' to the model due to incompatible shapes: (720,) in the checkpoint but (27,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
head.cls_score.{bias, weight}
The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


[04/05 21:59:35 d2.engine.train_loop]: Starting training from iteration 0


/home/koowater/.local/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/05 21:59:54 d2.utils.events]:  eta: 0:38:33  iter: 19  total_loss: 1.742  loss_cls: 1.37  loss_box_reg: 0.3777  time: 0.9427  data_time: 0.0139  lr: 1.9981e-05  max_mem: 3239M
[04/05 22:00:09 d2.utils.events]:  eta: 0:37:54  iter: 39  total_loss: 1.339  loss_cls: 1.068  loss_box_reg: 0.271  time: 0.8413  data_time: 0.0041  lr: 3.9961e-05  max_mem: 3239M
[04/05 22:00:25 d2.utils.events]:  eta: 0:37:39  iter: 59  total_loss: 1.109  loss_cls: 0.8988  loss_box_reg: 0.2281  time: 0.8110  data_time: 0.0041  lr: 5.9941e-05  max_mem: 3239M
[04/05 22:00:40 d2.utils.events]:  eta: 0:37:36  iter: 79  total_loss: 0.8292  loss_cls: 0.5917  loss_box_reg: 0.2223  time: 0.8006  data_time: 0.0043  lr: 7.9921e-05  max_mem: 3239M
[04/05 22:00:56 d2.utils.events]:  eta: 0:37:34  iter: 99  total_loss: 0.6639  loss_cls: 0.4603  loss_box_reg: 0.2306  time: 0.7966  data_time: 0.0042  lr: 9.9901e-05  max_mem: 3239M
[04/05 22:01:11 d2.utils.events]:  eta: 0:37:12  iter: 119  total_loss: 0.6654  loss_cls: 0.

In [6]:
from detectron2.engine import DefaultPredictor

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0001999.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [7]:
# 799
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("Duck-Farm-val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "Duck-Farm-val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`

[04/05 01:49:46 d2.data.datasets.coco]: Loaded 382 images in COCO format from ../datasets/Duck-Farm/annotations/labels_val.json
[04/05 01:49:46 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/05 01:49:46 d2.data.common]: Serializing 382 elements to byte tensors and concatenating them all ...
[04/05 01:49:46 d2.data.common]: Serialized dataset takes 0.20 MiB
[04/05 01:49:46 d2.evaluation.evaluator]: Start inference on 382 batches
[04/05 01:49:47 d2.evaluation.evaluator]: Inference done 11/382. Dataloading: 0.0005 s/iter. Inference: 0.0763 s/iter. Eval: 0.0001 s/iter. Total: 0.0770 s/iter. ETA=0:00:28
[04/05 01:49:52 d2.evaluation.evaluator]: Inference done 77/382. Dataloading: 0.0009 s/iter. Inference: 0.0750 s/iter. Eval: 0.0001 s/iter. Total: 0.0761 s/iter. ETA=0:00:23
[04/05 01:49:57 d2.evaluation.evaluator]: Inference done 143/382. Dataloading: 0.0009 s/iter. Infere